In [7]:
import openpyxl as xl
import pandas as pd
import os
from time import sleep
from datetime import datetime
import warnings

#Remover Avisos
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [3]:
files_dir = r"C:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Simuladores"
simuladorFiles = [f for f in os.listdir(files_dir) if f.endswith(".xlsm")]
simuladorFiles

['FI Simulador Juliana.xlsm',
 'FI Simulador Romilton.xlsm',
 'HE Simulador Agostinho.xlsm',
 'He Simulador Conrado.xlsm',
 'HE Simulador Margareide.xlsm',
 'HE Simulador Patricia.xlsm']

In [16]:
path = os.path.join(files_dir, simuladorFiles[2])
row2 = 2
simuladorFiles[2]

'HE Simulador Agostinho.xlsm'

In [17]:
wb = xl.load_workbook(path, data_only=True)
ws3 = wb['Simulador PDF']

In [24]:
#Coletar Informações do Quadro Simulação
amortizacao = None
indexador = None
prazo = None
carencia = None
pularParcela = None
diaVencimento = None
taxaAM = None
CET = None

#Coletar Informações do Quadro Resultados
LTV = None
IOF = None
TAC = None
registro = None
valorBruto = None
valorLiquido = None
valorAvaliacao = None
primeiraParcela = None

#Auxiliar Primeira Parcela
auxPar = False
dataPriParcela = None
dataDesembolco = None
priParcelaAmortizao = None
priParcelaJuros = None
dataUltimaParcela = 0
saldoDevedor = None
auxSaldo = False


for col in ws3.iter_cols(min_col=1,max_col=15, min_row = 1, max_row=100):
    for cell in col:
        if cell.value == 'Simulação':
            #Definir variáveis auxiliares de linha e coluna
            rowAux = cell.row + 1
            columnAux = cell.column + 3

            #Extração da tabela simulação
            amortizacao = ws3.cell(row=rowAux, column=columnAux).value
            indexador = ws3.cell(row=rowAux+1, column=columnAux).value
            prazo = ws3.cell(row=rowAux+2, column=columnAux).value
            carencia = ws3.cell(row=rowAux+3, column=columnAux).value
            pularParcela = ws3.cell(row=rowAux+4, column=columnAux).value
            diaVencimento = ws3.cell(row=rowAux+5, column=columnAux).value
            taxaAM = ws3.cell(row=rowAux+6, column=columnAux).value
            CET = ws3.cell(row=rowAux+7, column=columnAux).value
        
        elif cell.value == 'Resultados':
            #Definir variáveis auxiliares de linha e coluna
            rowAux = cell.row + 1
            columnAux = cell.column + 3

            #Extralção da tabela Resultados
            LTV = ws3.cell(row=rowAux, column=columnAux).value
            IOF = ws3.cell(row=rowAux+1, column=columnAux).value
            TAC = ws3.cell(row=rowAux+2, column=columnAux).value
            registro = ws3.cell(row=rowAux+3, column=columnAux).value
            valorBruto = ws3.cell(row=rowAux+4, column=columnAux).value
            valorLiquido = ws3.cell(row=rowAux+5, column=columnAux).value
            valorAvaliacao = ws3.cell(row=rowAux+6, column=columnAux).value
            primeiraParcela = ws3.cell(row=rowAux+7, column=columnAux).value
        
        elif cell.value == 'Data':
            rowAux = cell.row + 1
            columnAux = cell.column + 1
            dataDesembolco = ws3.cell(row=rowAux, column=cell.column).value
            for i in range(1,10):
                parcelaMensal = ws3.cell(row=rowAux+i, column=columnAux).value
                if parcelaMensal > 0 and auxPar == False:
                    priParcelaAmortizao = ws3.cell(row=rowAux+i, column=columnAux+1).value
                    priParcelaJuros = ws3.cell(row=rowAux+i, column=columnAux+2).value
                    dataPriParcela = ws3.cell(row=rowAux+i, column=columnAux-1).value
                    #print(f"Parcela do mês: {parcelaMensal} \nAmortização: {priParcelaAmortizao}\nJuros: {priParcelaJuros}\nAmortização+Juros: {str(priParcelaJuros+priParcelaAmortizao)}")
                    auxPar = True
        
        elif cell.value == 'Saldo Devedor':
            for i in range(1,400):
                saldoDevedor = ws3.cell(row=rowAux+i, column=cell.column).value
                try:
                    if saldoDevedor <= 10 and auxSaldo ==False:
                        dataUltimaParcela = ws3.cell(row=rowAux+i, column=cell.column-4).value
                        auxSaldo = True
                except:
                    pass
amortizacaoEJuros = priParcelaAmortizao + priParcelaJuros
prestacoes = prazo - carencia
nomeCliente = ws3.cell(row=1, column=7).value
wb.close()

**Conferência dos Valores Extraídos**

In [25]:
# #Printar Informações do Quadro Simulação
print(f"Valores Extraídos do quado de Simulação")
print(f"Amoritzação: {amortizacao} \nIndexador: {indexador} \nPrazo: {prazo}\nCarência: {carencia}\nPular Parcela: {pularParcela}\nDia de Vencimento: {diaVencimento}\nTaxa ao mês {taxaAM}\nCET: {CET}")

# #Printar Informações do Quadro Resultados
print(f"\n\nValores Extraídos do quado de Resultados")
print(f"Data Desembolço: {dataDesembolco}\nLTV: {LTV}\nIOF: {IOF}\nTAC: {TAC}\nRegistro: {registro}\nValor Bruto: {valorBruto}\nValor Líquido: {valorLiquido}\nValor de Avaliação: {valorAvaliacao}\nValor Primeira Parcela: {primeiraParcela}")

# #Auxiliar Primeira Parcela
print(f"\n\nValores Extraídos da Primeira Parcela")
print(f"Valor de Amortização: {priParcelaAmortizao}\nValor do Juros: {priParcelaJuros}\nAmortização+Juros: {str(priParcelaJuros+priParcelaAmortizao)}\nData Primeira Parcela: {dataPriParcela}\nData Última Parcela: {dataUltimaParcela}")

print(f"Prazo: {prestacoes}")

Valores Extraídos do quado de Simulação
Amoritzação: SAC 
Indexador: IPCA 
Prazo: 228
Carência: 6
Pular Parcela: Nenhum
Dia de Vencimento: 30
Taxa ao mês 0.0092
CET: 0.14409638500653799


Valores Extraídos do quado de Resultados
Data Desembolço: 2022-06-08 00:00:00
LTV: 0.49264601449921586
IOF: 6224.231451705161
TAC: 6768
Registro: 3042.67
Valor Bruto: 185234.90145170517
Valor Líquido: 169200
Valor de Avaliação: 376000
Valor Primeira Parcela: 2800.520024418839


Valores Extraídos da Primeira Parcela
Valor de Amortização: 889.9910082772791
Valor do Juros: 1817.7176353356729
Amortização+Juros: 2707.708643612952
Data Primeira Parcela: 2023-01-30 00:00:00
Data Última Parcela: 2041-06-30 00:00:00
Prazo: 222


In [149]:
keyValue = {'valorTotal':round(valorBruto,2),'tabela': amortizacao,'registro': round(registro,2),'taxaAM':taxaAM,'valorLiquido': round(valorLiquido,2),'prazoMes': prazo,
                'valorPrimeiraParcelaComEncargos':round(primeiraParcela,2), 'valorImóvel':round(valorAvaliacao,2),'prazoContrato': prestacoes,'ultimaParcela':dataUltimaParcela.strftime("%d/%m/%Y"),
                'dataContrato': dataDesembolco.strftime("%d/%m/%Y"), 'valorPrimeiraParcela': round(amortizacaoEJuros,2), 'primeiraParcela':dataPriParcela.strftime("%d/%m/%Y"),'indice': indexador
                }

In [150]:
keyValue

{'valorTotal': 433835.93,
 'tabela': 'Price',
 'registro': 3240.05,
 'taxaAM': 0.0107,
 'valorLiquido': 400000,
 'prazoMes': 240,
 'valorPrimeiraParcelaComEncargos': 5399.79,
 'valorImóvel': 930000,
 'prazoContrato': 237,
 'ultimaParcela': '04/07/2042',
 'dataContrato': '05/07/2022',
 'valorPrimeiraParcela': 5215.54,
 'primeiraParcela': '04/11/2022',
 'indice': 'IPCA'}

In [133]:
dataPriParcela.strftime("%d/%m/%Y")


'15/09/2022'